In [3]:
# ===============================================================================================
# @des: Ensemble Kalmer Filter: We will later integrate this EnKF with several ICE sheet models.
# @date: 12-09-2024
# @author: Brian KYANJO
# ===============================================================================================

In [4]:
# import libraries --------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt


## Steps to follow
- Start with replicating the Julia Code in Python
- This will help in comparing results later 
- After when python agrees with Julia
- Form General Classes for the code to make it more genral using OBP
- Profile the code to see how long the filter takes for very big vlaues of N
- Rewrite the Filter to C and call it in python 
- if Still slow, add parallel capabilities using MPI or OpenMP and still wrap it in python.
- Once this works, now proceed to trying to couple the filter with another problem
- If sucess then, now try with an actual ICE-sheet model
- Continue to remodel and develop the wrapper ....
- Finnally convert the wrapper into a python script

In [ ]:
# Parameters definition

def params_define():
    # Prescribed initial values of model parameters in a dictionary
    params = {}
    params["A"]         = 4e-26         
    params["year"]      = 3600 * 24 * 365
    params["n"]         = 3
    params["C"]         = 3e6
    params["rho_i"]     = 900
    params["rho_w"]     = 1000
    params["g"]         = 9.8
    params["B"]         = params["A"] ** (-1 / params["n"])
    params["m"]         = 1 / params["n"]
    params["accum"]     = 0.65 / params["year"]
    params["facemelt"]  = 5 / params["year"]

    # Scaling parameters
    params["hscale"]    = 1000
    params["ascale"]    = 1.0 / params["year"]
    params["uscale"]    = (params["rho_i"] * params["g"] * params["hscale"] * params["ascale"] / params["C"]) ** (1 / (params["m"] + 1))
    params["xscale"]    = params["uscale"] * params["hscale"] / params["ascale"]
    params["tscale"]    = params["xscale"] / params["uscale"]
    params["eps"]       = params["B"] * ((params["uscale"] / params["xscale"]) ** (1 / params["n"])) / (2 * params["rho_i"] * params["g"] * params["hscale"])
    params["lambda"]    = 1 - (params["rho_i"] / params["rho_w"])

    # Grid parameters
    params["NT"]        = 1
    params["TF"]        = params["year"]
    params["dt"]        = params["TF"] / params["NT"]
    params["transient"] = 0
    params["tcurrent"]  = 1

    params["N1"]        = 40
    params["N2"]        = 10
    params["sigGZ"]     = 0.97
    params["NX"]        = params["N1"] + params["N2"]

    # Bed params
    # params["b0"] = -400
    params["xsill"]     = 50e3
    params["sillamp"]   = 500
    params["sillsmooth"] = 1e-5
    # params["bxr"] = 1e-3
    # params["bxp"] = -1e-3

    # EnKF params
    params["inflation"] = 1.0
    params["assim"]     = False

    # Generating sigma values
    sigma1    = np.linspace(params["sigGZ"] / (params["N1"] + 0.5), params["sigGZ"], int(params["N1"]))
    sigma2    = np.linspace(params["sigGZ"], 1, int(params["N2"] + 1))
    sigma     = np.concatenate((sigma1, sigma2[1:params["N2"] + 1]))

    # Create the grid dictionary
    grid                = {"sigma": sigma}
    grid["sigma_elem"]  = np.concatenate(([0], (sigma[:-1] + sigma[1:]) / 2))
    grid["dsigma"]      = np.diff(grid["sigma"])

    return params, grid

In [ ]:
# Observation operator ----------------------
def Obs(huxg_virtual_obs,m_obs):
    n = huxg_virtual_obs.shape[0]
    m = m_obs
    H = np.zeros((m*2+1,n))
    di = int((n-2)/(2*m))  #distance between measurements
    for i in range(1,m+1):
        H[i-1,i*di] = 1
        H[m+i-1,int(((n-2)/2)+(i*di))] = 1
    H[m*2+1,n-1] = 1
    z = H.dot(huxg_virtual_obs)
    return z

In [ ]:
# Jacobian of the observation operator
def JObs(n_model, m_obs):
    n = n_model
    m = m_obs
    H = np.zeros((m*2+1, n))  # Create an array of zeros
    di = int((n - 2) / (2 * m))  # Calculate the distance between measurements

    for i in range(1, m+1):
        H[i-1, i*di] = 1  # Assign 1 to appropriate indices
        H[m + i - 1, int((n - 2) / 2 + i * di)] = 1
    
    H[m*2, n-1] = 1  # Set the last element in the matrix
    return H


In [ ]:
# EnKF function ------------------------------
def EnKF(huxg_ens,huxg_obs,ObsFun,JObsFun,Cov_obs,Cov_model,params):
    n = huxg_ens.shape[0]  # State size
    N = huxg_ens.shape[1]  # Ensemble size
    m = huxg_obs.shape[0]  # Measurement size

    huxg_ens_mean = np.mean(huxg_ens, axis=1)   # Mean of model forecast ensemble
    Jobs = JObsFun(n, params["m_obs"])          # Jacobian of observation operator

    KalGain = Cov_model @ Jobs.T @ np.linalg.inv(Jobs @ Cov_model @ Jobs.T + Cov_obs)  # Compute Kalman gain

    obs_virtual = np.zeros((m, N))
    analysis_ens = np.zeros((n, N))
    # mismatch_ens = np.zeros((n-1, N))
    for i in range(N):
        obs_virtual[:, i] = huxg_obs + np.random.multivariate_normal(np.zeros(m), Cov_obs)  # Generate virtual observations
        analysis_ens[:, i] = huxg_ens[:, i] + KalGain @ (obs_virtual[:, i] - ObsFun(huxg_ens[:, i], params["m_obs"]))  # Generate analysis ensemble
        # mismatch_ens[:, i] = obs_virtual[:, i] - ObsFun(huxg_ens[:, i], params["m_obs"])  # Calculate mismatch

    analysis_ens_mean = np.mean(analysis_ens, axis=1)  # Mean of analysis ensemble

    analysis_cov = (1 / (N - 1)) * (analysis_ens - np.repeat(analysis_ens_mean, N, axis=1)) @ (analysis_ens - np.repeat(analysis_ens_mean, N, axis=1)).T  # Analysis error covariance
    analysis_cov = analysis_cov * taper
    return analysis_ens, analysis_cov
    

In [1]:
# # The Ensemble Kalman Filter (EnKF) class ---------------------------------------
# class EnKF:
#     '''
#     Parameters:
#     -----------
#     n = number of state variables
#     m = number of observations
#     R = observation error covariance matrix
#     dt = time step
#     Q = process noise covariance matrix
#     P = state covariance matrix
#     x = state vector
#     X = state ensemble
#     Y = observation ensemble
#     K = Kalman gain
#     y = observation vector
#     yo = observation mean
#     xo = state mean
#     '''
#     def __init__(self, model, n, m, R, dt):
#         self.model = model
#         self.n = n
#         self.m = m
#         self.R = R
#         self.dt = dt
#         self.Q = np.eye(n) * 1e-5
#         self.P = np.eye(n)
#         self.x = np.random.randn(n)
#         self.X = np.random.randn(n, n)
#         self.Y = np.random.randn(m, n)
#         self.K = np.zeros((n, m))
#         self.y = np.zeros(m)
#         self.yo = np.zeros(m)
#         self.xo = np.zeros(n)

#     def forecast(self):
#         self.X = self.model(self.X, self.dt) + np.random.multivariate_normal(np.zeros(self.n), self.Q, self.n).T

#     def analysis(self, y):
#         self.yo = y
#         self.xo = np.mean(self.X, axis=1)
#         self.Y = self.model(self.X, self.dt)
#         self.y = np.mean(self.Y, axis=1)
#         Pyy = np.cov(self.Y) + self.R
#         Pxy = np.cov(self.X, self.Y)
#         self.K = np.dot(Pxy, np.linalg.inv(Pyy))
#         self.X = self.X + np.dot(self.K, (y - self.y))
#         self.P = np.cov(self.X)

